In [266]:
import pandas as pd
import numpy as np

In [267]:
df_user_log = pd.read_csv("data_set/user_behavior_time_resampled.csv")
df_user_info = pd.read_csv("data_set/user_info.csv")



In [278]:
df_vip_user = pd.read_csv("data_set/vip_user.csv")

In [269]:
# 分别插入 click、cart、order、fav 四列，表示点击、加购物车、下单和收藏
# 规则见上面的文字描述
df_user_log["click"] = df_user_log.action_type.apply(lambda l:1 if l==0 else 0)
df_user_log["cart"] = df_user_log.action_type.apply(lambda l:1 if l==1 else 0)
df_user_log["order"] = df_user_log.action_type.apply(lambda l:1 if l==2 else 0)
df_user_log["fav"] = df_user_log.action_type.apply(lambda l:1 if l==3 else 0)

# 查看添加之后的 user_log 表
df_user_log

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,timestamp,click,cart,order,fav
0,328862,323294,833,2882,2661.0,829,0,62242.300959,1,0,0,0
1,328862,623866,1271,2882,2661.0,829,0,75598.841378,1,0,0,0
2,328862,239288,602,420,4953.0,801,0,69606.479645,1,0,0,0
3,328862,197239,602,420,4953.0,801,0,62758.912202,1,0,0,0
4,328862,524981,664,2382,1272.0,602,0,39651.123222,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10985061,208016,869502,898,3716,5508.0,1110,0,58128.784724,1,0,0,0
10985062,208016,51712,898,3763,8434.0,1110,0,84907.294322,1,0,0,0
10985063,208016,952056,898,3763,8434.0,1110,0,64118.247166,1,0,0,0
10985064,208016,952056,898,3763,8434.0,1110,0,18877.391838,1,0,0,0


In [271]:
print("click 数据分布：\n",df_user_log.click.value_counts())
print("order 数据分布：\n",df_user_log.order.value_counts())
print("fav 数据分布：\n",df_user_log.fav.value_counts())
print("cart 数据分布：\n",df_user_log.cart.value_counts())

click 数据分布：
 1    9709458
0    1275608
Name: click, dtype: int64
order 数据分布：
 0    10325489
1      659577
Name: order, dtype: int64
fav 数据分布：
 0    10384328
1      600738
Name: fav, dtype: int64
cart 数据分布：
 0    10969773
1       15293
Name: cart, dtype: int64


In [272]:
df_clean = df_user_log[["item_id", "click", "cart", "order","fav"]]
df_clean


,item_id,click,cart,order,fav
0,323294,1,0,0,0
1,623866,1,0,0,0
2,239288,1,0,0,0
3,197239,1,0,0,0
4,524981,1,0,0,0
...,...,...,...,...,...
10985061,869502,1,0,0,0
10985062,51712,1,0,0,0
10985063,952056,1,0,0,0
10985064,952056,1,0,0,0


In [273]:
df_item = df_clean.groupby(["item_id"]).sum()
df_item

,click,cart,order,fav
item_id,,,,
2,511,0,67,60
3,22,0,2,1
4,5,0,0,0
7,1,0,0,0
8,11,0,3,0
...,...,...,...,...
1113160,0,0,1,0
1113161,1,0,0,0
1113162,17,0,2,1


In [279]:
df_vip_user

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0
...,...,...,...
260859,359807,4325,0
260860,294527,3971,0
260861,294527,152,0
260862,294527,2537,0


In [280]:
# 重命名 merchant_id 为 seller_id
df_vip_user = df_vip_user.rename( columns={"merchant_id" : "seller_id"})

# 选取 seller_id和label 这连烈，斌将结果通过 seller_id 聚合，相同 seller_id 的记录的label求和
df_brand_vip_users = df_vip_user[["seller_id", "label"]].groupby("seller_id").sum()

# 查看保存了店铺 vip 用户数的表的内容
df_brand_vip_users


,label
seller_id,
2,2
8,1
9,1
10,16
13,15
...,...
4987,0
4988,2
4991,0


In [281]:
# 筛选出 seller_id 和 item_id
df_seller_item_count = df_user_log[["seller_id", "item_id"]]

# 按item_id 去重，因为 itemid 一样的记录，seller_id 肯定也一样，所以 seller_id没有影响
df_seller_item_count = df_seller_item_count.drop_duplicates("item_id")

# 将 item_id 列赋值为 1，方便做求和
df_seller_item_count["item_id"] = 1

# 按seller_id 聚合，然后针对 item_id 列求和
df_seller_item_count = df_seller_item_count.groupby("seller_id").sum()

# 将 item_id 列改为 item_count， 避免有歧义
df_seller_item_count = df_seller_item_count.rename(columns = {"item_id" : "item_count"})

# 查看结果
df_seller_item_count

,item_count
seller_id,
1,2547
2,96
3,87
4,96
5,309
...,...
4991,14
4992,87
4993,147


In [282]:
df_item

,click,cart,order,fav
item_id,,,,
2,511,0,67,60
3,22,0,2,1
4,5,0,0,0
7,1,0,0,0
8,11,0,3,0
...,...,...,...,...
1113160,0,0,1,0
1113161,1,0,0,0
1113162,17,0,2,1


In [283]:
# 从原始行为表中取出 item_id 和seller_id ，构成新表
df_brand_item_map = df_user_log[["item_id", "seller_id"]]

# 按照item_id去重，去重后得到的结果就相当于是 item id 和 seller_id 的映射关系
df_brand_item_map = df_brand_item_map.drop_duplicates("item_id")

# 将 df_brand_item_map 映射进 df_item， 以 item_id 为 key
df_item = df_item.merge(df_brand_item_map, how="left", on="item_id")

# 查看最新的商品特征表
df_item

,item_id,click,cart,order,fav,seller_id
0,2,511,0,67,60,1381
1,3,22,0,2,1,2705
2,4,5,0,0,0,4007
3,7,1,0,0,0,1640
4,8,11,0,3,0,2592
...,...,...,...,...,...,...
756720,1113160,0,0,1,0,3825
756721,1113161,1,0,0,0,1543
756722,1113162,17,0,2,1,2586
756723,1113163,5,0,0,1,2654


In [231]:
df_brand_item_map

,item_id,seller_id
0,323294,2882
1,623866,2882
2,239288,420
3,197239,420
4,524981,2382
...,...,...
10984571,704621,2151
10984740,227512,3019
10984827,226104,705
10984906,184975,4183


In [284]:
# 将店铺VIP用户特征表拼接到商品特征表中，以 seller_id 为 key
df_item = df_item.merge(df_brand_vip_users,how = "left", on="seller_id")
df_item


,item_id,click,cart,order,fav,seller_id,label
0,2,511,0,67,60,1381,NaN
1,3,22,0,2,1,2705,NaN
2,4,5,0,0,0,4007,3.0
3,7,1,0,0,0,1640,2.0
4,8,11,0,3,0,2592,16.0
...,...,...,...,...,...,...,...
756720,1113160,0,0,1,0,3825,1.0
756721,1113161,1,0,0,0,1543,0.0
756722,1113162,17,0,2,1,2586,NaN
756723,1113163,5,0,0,1,2654,3.0


In [285]:
# 用 0 填充缺失值
df_item["label"] = df_item["label"].fillna(0)
df_item

,item_id,click,cart,order,fav,seller_id,label
0,2,511,0,67,60,1381,0.0
1,3,22,0,2,1,2705,0.0
2,4,5,0,0,0,4007,3.0
3,7,1,0,0,0,1640,2.0
4,8,11,0,3,0,2592,16.0
...,...,...,...,...,...,...,...
756720,1113160,0,0,1,0,3825,1.0
756721,1113161,1,0,0,0,1543,0.0
756722,1113162,17,0,2,1,2586,0.0
756723,1113163,5,0,0,1,2654,3.0


In [235]:
df_item.isnull().sum()
df_item = df_item.dropna()

In [299]:
df_item = df_item.merge(df_seller_item_count, how = "left", on = "seller_id")
df_item

,item_id,click,cart,order,fav,seller_id,label,item_count
0,2,511,0,67,60,1381,0.0,181
1,3,22,0,2,1,2705,0.0,70
2,4,5,0,0,0,4007,3.0,16
3,7,1,0,0,0,1640,2.0,80
4,8,11,0,3,0,2592,16.0,197
...,...,...,...,...,...,...,...,...
756720,1113160,0,0,1,0,3825,1.0,461
756721,1113161,1,0,0,0,1543,0.0,298
756722,1113162,17,0,2,1,2586,0.0,175
756723,1113163,5,0,0,1,2654,3.0,411


In [298]:
df_item

,item_id,click,cart,order,fav,seller_id,label
0,2,511,0,67,60,1381,0.0
1,3,22,0,2,1,2705,0.0
2,4,5,0,0,0,4007,3.0
3,7,1,0,0,0,1640,2.0
4,8,11,0,3,0,2592,16.0
...,...,...,...,...,...,...,...
756720,1113160,0,0,1,0,3825,1.0
756721,1113161,1,0,0,0,1543,0.0
756722,1113162,17,0,2,1,2586,0.0
756723,1113163,5,0,0,1,2654,3.0


In [310]:
# 导入分割的方法
from sklearn.model_selection import train_test_split

# 自变量的数据表
X = df_item.drop(columns=["order", "seller_id","item_id","label", "item_count"])

# 因变量
y = df_item["order"]

# 分别切割出训练集，测试集，测试集的比例是 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

# 查看自变量的测试集合
X_train


,click,cart,fav
260577,1,0,0
314027,2,0,0
154884,18,0,1
515741,24,0,1
18269,2,0,1
...,...,...,...
259178,2,0,0
365838,3,0,0
131932,13,0,0
671155,22,0,0


,click,cart,fav,label
260577,1,0,0,3.0
314027,2,0,0,0.0
154884,18,0,1,2.0
515741,24,0,1,207.0
18269,2,0,1,29.0
...,...,...,...,...
259178,2,0,0,0.0
365838,3,0,0,0.0
131932,13,0,0,0.0
671155,22,0,0,0.0


In [261]:
y

0         67
1          2
2          0
3          0
4          3
          ..
756720     1
756721     0
756722     2
756723     0
756724     5
Name: order, Length: 756725, dtype: int64

In [297]:
X_train

,click,cart,fav,label
260577,1,0,0,3.0
314027,2,0,0,0.0
154884,18,0,1,2.0
515741,24,0,1,207.0
18269,2,0,1,29.0
...,...,...,...,...
259178,2,0,0,0.0
365838,3,0,0,0.0
131932,13,0,0,0.0
671155,22,0,0,0.0


In [263]:
X_train.isnull().sum()

click         0
cart          0
fav           0
label         0
item_count    0
dtype: int64

In [311]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression()

In [312]:
# predict X_test 对应的 y 值
y_pred = linear_model.predict(X_test)

# 用 scores 方法，查看模型自变量和因变量的相关性
print("Scores:", linear_model.score(X_test, y_test))

# 查看 predict y 和 y test的平均绝对误差
print("MAE:", mean_absolute_error(y_test, y_pred))

# 查看模型的 b 值
print("intercept:", linear_model.intercept_)

# 查看模型的系数
print("coef_:", linear_model.coef_)

Scores: 0.439706935083249
MAE: 0.9521912431626115
intercept: 0.15885011639844204
coef_: [-1.08757222e-03  7.66812082e+00  7.19513463e-01]


In [309]:
print(linear_model.predict([[20, 0 ,3, 10, 30]]))

[2.46262277]
